In [ ]:
%reload_ext autoreload
%autoreload 2

# All the packages are defined in the Config file
from model_packages import *

# All the variables are defined in the Config file
from model_config import *

# All the functions are defined in the Config file
from model_utils import *





In [ ]:
# Preprocess Strava Metro data
# Get hourly means for weekdays in non-winter months

#os.getcwd()

# we have made a local copy of the Strava data
'''
The data is for a specific 
location with hourly 
frequency count of activities
'''

#folder_list=os.listdir(strava_data_loc_np)


folder_list=[name for name in os.listdir(strava_data_loc_np) if os.path.isdir(os.path.join(strava_data_loc_np, name)) ]


str_csv=[]

str_shp=[]
for folder in folder_list:
    
    file_list=os.listdir(strava_data_loc_np+folder)
    print(folder)
    
    csv_file=[x for x in file_list if x.endswith(".csv")][0]
    
    shp_file=[x for x in file_list if x.endswith(".shp")][0]
    
    # Shape files and Strava edge ids.
    strava_df_shp = gpd.read_file(strava_data_loc_np+folder+'/'+shp_file)
    
    strava_df_shp=strava_df_shp.drop_duplicates(subset='edgeUID').reset_index(drop=True)
    
    # Strava activity count
    strava_df_csv = pd.read_csv(strava_data_loc_np+folder+'/'+csv_file)
    strava_df_shp=strava_df_shp.to_crs(crs_mtr)
    
    str_csv.append(strava_df_csv)
    str_shp.append(strava_df_shp)

In [ ]:
strava_df_csv=pd.concat(str_csv).drop_duplicates().reset_index(drop=True)


strava_df_shp=pd.concat(str_shp).drop_duplicates(subset=['edgeUID']).reset_index(drop=True)

In [ ]:



strava_df_csv['date']=pd.to_datetime(strava_df_csv['date'])

In [ ]:
strava_df_csv['day_of_week'] = strava_df_csv['date'].dt.day_name()

In [ ]:

# Get a sample of Strava activities: (need to find a way around- to speed up computation)

strava_df_csv_sample=strava_df_csv.copy()

In [ ]:




#strava_df_csv_sample.date=pd.to_datetime(strava_df_csv_sample.date)


strava_df_csv_sample['month']=pd.to_datetime(strava_df_csv_sample['date']).dt.month

strava_df_csv_sample['year']=pd.to_datetime(strava_df_csv_sample['date']).dt.year

#strava_df_csv_sample.month=pd.to_datetime(strava_df_csv_sample.month)

#strava_df_csv_sample['year']=strava_df_csv_sample.month.dt.year


#strava_df_csv_sample['month']=strava_df_csv_sample.month.dt.month



In [ ]:
seasons = {(1, 12, 2): 'Winter', (3, 4, 5): 'Spring', (6, 7, 8): 'Summer', (9, 10, 11): 'Autumn'}


def get_season(ser):
    for k in seasons.keys():
        if ser in k:
            return seasons[k]

strava_df_csv_sample['season'] = strava_df_csv_sample.month.apply(get_season)

In [ ]:
strava_df_shp.to_crs(crs_deg).explore()

In [ ]:
from pandas.api.types import CategoricalDtype

cats = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
cat_type = CategoricalDtype(categories=cats, ordered=True)
strava_df_csv_sample['day_of_week'] = strava_df_csv_sample['day_of_week'].astype(cat_type)



#df_agg_day_sesn=strava_df_csv_sample.groupby(['day_of_week','season'])['total_trip_count'].sum().reset_index()
df_agg_day_sesn=((strava_df_csv_sample.groupby(['season','day_of_week'])['total_trip_count'].sum()/\
 strava_df_csv_sample.groupby(['season','day_of_week'])['total_trip_count'].sum().sum(axis=0))*100).reset_index()

In [ ]:


fig = px.bar(df_agg_day_sesn, x='day_of_week', y="total_trip_count", color="season",barmode='group')#,markers=True,symbol="season")

fig.update_traces(marker=dict(size=15,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.update_layout(yaxis_title="% Total recorded Strava activities")

fig.show()

In [ ]:
#ax=strava_df_shp.to_crs(crs_deg).plot()
#cx.add_basemap(ax,crs=strava_df_shp.to_crs(crs_deg).crs.to_string())

In [ ]:


#nature_reserve=gpd.read_file(nature_reserve_data)



#nature_reserve.geometry = nature_reserve.to_crs(crs_mtr).geometry.buffer(100, 6)

In [ ]:
#strava_df_shp['osmId'].unique().shape

strava_df_csv_sample['osm_reference_id'].unique().shape

In [ ]:
#nature_reserve.explore()

In [ ]:
national_park=gpd.read_file(national_park_data)

In [ ]:
#ax=nature_reserve.to_crs(crs_deg).plot()
#cx.add_basemap(ax,crs=strava_df_shp.to_crs(crs_deg).crs.to_string())

In [ ]:
national_park.explore(categorical=True,column='name',legend=True)

In [ ]:


#ax=gpd.overlay(strava_df_shp.to_crs(crs_mtr), nature_reserve.to_crs(crs_mtr), how='intersection').\
#to_crs(crs_deg).plot(categorical=True,column='nnr_name',legend=True)
#cx.add_basemap(ax,crs=strava_df_shp.to_crs(crs_deg).crs.to_string())

In [ ]:
gpd.overlay(strava_df_shp.to_crs(crs_mtr), national_park.to_crs(crs_mtr), how='intersection').\
to_crs(crs_deg).explore(categorical=True,column='name',legend=True)

In [ ]:
edges_nature_rsrv=gpd.overlay(strava_df_shp.to_crs(crs_mtr), national_park.to_crs(crs_mtr),\
                              how='intersection')['edgeUID'].unique()



strava_df_csv_sample=strava_df_csv_sample[strava_df_csv_sample['edge_uid'].isin(edges_nature_rsrv)]



In [ ]:
strava_df_shp=strava_df_shp[strava_df_shp['edgeUID'].isin(edges_nature_rsrv)]

In [ ]:

strava_df_csv_sample['month_name']=strava_df_csv_sample['month'].apply(lambda x: calendar.month_abbr[x])


In [ ]:
# Visualisation
strava_df_csv_sample.groupby(['month','year'])['total_trip_count'].mean().unstack().plot(style='-o')
plt.title('Total number of trips recorded across all edges')

In [ ]:
# Focus on specific days and months


strava_df_csv_sample_summer=strava_df_csv_sample[strava_df_csv_sample.month_name.isin(['Jun','Jul','Aug'])]

In [ ]:
strava_df_csv_sample_summer['month_name'].unique()

In [ ]:
# Visualisation
strava_df_csv_sample_summer.groupby(['month','year'])['total_trip_count'].mean().unstack().plot(style='-o')
#plt.ylabel('Activities recorded')
plt.title('Total number of trips recorded across all edges during Summer')

In [ ]:
strava_df_csv_sample_summer

In [ ]:
# Mean activity count of each edge grouped by month

mean_count=strava_df_csv_sample_summer.groupby(['month','edge_uid'])['total_trip_count'].mean().\
reset_index()



mean_count_pvt_tbl=mean_count.pivot_table(index=["edge_uid"],columns='month',values='total_trip_count')


mean_count_pvt_tbl=mean_count_pvt_tbl.fillna(0)

mean_count_pvt_tbl#=mean_count_pvt_tbl.reindex(columns=['Morning', 'Noon', 'Evening', 'Night'])

In [ ]:

sns.displot(pd.DataFrame(mean_count_pvt_tbl.sum(axis=1)).values,kde=True)

In [ ]:
mean_count_pvt_tbl.values


In [ ]:
# Calculate the pairwise distance between edges and 

ds = dtw.distance_matrix_fast(mean_count_pvt_tbl.values,window=1,compact=False)


In [ ]:
# Based on the pairwise distance, assign each edge to a cluster
'''
This is computationally costly
'''
# https://stackoverflow.com/questions/50695226/how-to-get-the-optimal-number-of-clusters-using-hierarchical-cluster-analysis-au

# costly
#cluster = AgglomerativeClustering(n_clusters=None, affinity='euclidean', linkage='ward',\
#                                  compute_full_tree=True, distance_threshold=10000)

# racplus
# https://towardsdatascience.com/scaling-agglomerative-clustering-for-big-data-an-introduction-to-rac-fb26a6b326ad


numbr_clustrs_range=range (2,25)
str_chs=[]
for numb_clusters in numbr_clustrs_range:
    
    cluster = AgglomerativeClustering(n_clusters=numb_clusters, affinity='euclidean', linkage='ward',\
                                  compute_full_tree=True, distance_threshold=None)

#cluster = racplusplus.rac(n_clusters=5,distance_threshold=None, symmetric, batch_size=1000, no_cores=8, metric="cosine")


# https://hdbscan.readthedocs.io/en/latest/performance_and_scalability.html
#cluster= HDBSCAN()
# Cluster the data
    cluster.fit_predict(ds)

    print(f"Number of clusters = {1+np.amax(cluster.labels_)}")

# Display the clustering, assigning cluster label to every datapoint 
#print("Classifying the points into clusters:")
#print(cluster.labels_)

# Display the clustering graphically in a plot
#plt.scatter(ds[:,0],ds[:,1], c=cluster.labels_, cmap='rainbow')
#plt.title(f"SK Learn estimated number of clusters = {1+np.amax(cluster.labels_)}")
#plt.show()

# https://www.geeksforgeeks.org/calinski-harabasz-index-cluster-validity-indices-set-3/

    metrics.calinski_harabasz_score(ds, cluster.labels_)
    
    str_chs.append(metrics.calinski_harabasz_score(ds, cluster.labels_))

    #print(" ")
    
plt.plot(numbr_clustrs_range,str_chs,'-o')




In [ ]:


cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward',\
                                  compute_full_tree=True, distance_threshold=None)

#cluster = racplusplus.rac(n_clusters=5,distance_threshold=None, symmetric, batch_size=1000, no_cores=8, metric="cosine")


# https://hdbscan.readthedocs.io/en/latest/performance_and_scalability.html
#cluster= HDBSCAN()
# Cluster the data
cluster.fit_predict(ds)

print(f"Number of clusters = {1+np.amax(cluster.labels_)}")

In [ ]:

# Assign cluster label to each edge
df_ds=pd.DataFrame(ds)

df_ds.index=mean_count_pvt_tbl.index.values

df_ds['total_mean_edge_count']=df_ds.mean(axis=1).values

df_ds['labels']=cluster.labels_

df_ds=df_ds.reset_index()

df_ds.rename(columns={'index':'edge_uid'},inplace=True)


count_edges_clusters=df_ds[['edge_uid','labels','total_mean_edge_count']].merge(mean_count_pvt_tbl.reset_index(),left_on=['edge_uid'],\
                                 right_on=['edge_uid'])


count_edges_clusters=count_edges_clusters[['edge_uid','labels','total_mean_edge_count']]





In [ ]:
strava_df_shp
df_ds
count_edges_clusters

In [ ]:

# Strava edges with cluster labels

df_ds_shp=pd.merge(strava_df_shp.drop_duplicates(subset=['edgeUID']),df_ds,left_on=['edgeUID'],\
                   right_on=['edge_uid'],how='inner')

df_ds_shp=gpd.GeoDataFrame(df_ds_shp)[['geometry','labels','edge_uid']]


df_ds_shp['labels_count']=df_ds_shp['labels'].map(df_ds_shp['labels'].value_counts())

In [ ]:
count_edges_clusters=df_ds_shp.merge(count_edges_clusters,left_on=['edge_uid','labels'],\
                right_on=['edge_uid','labels'])

In [ ]:
#ax=count_edges_clusters.to_crs(crs_deg).plot()
#national_park.to_crs(crs_deg).plot(color='g',ax=ax)
#cx.add_basemap(ax,crs=count_edges_clusters.to_crs(crs_deg).crs.to_string())

In [ ]:

sns.violinplot(data=count_edges_clusters,x="labels",y="total_mean_edge_count")

In [ ]:
#dict_label={0:'high_footfall',1:'low_footfall',2:'medium_footfall'}

In [ ]:

sns.violinplot(data=count_edges_clusters,x="labels",y="total_mean_edge_count")

In [ ]:
#count_edges_clusters['labels']=count_edges_clusters['labels'].map(dict_label)

In [ ]:
sns.violinplot(data=count_edges_clusters,x="labels",y="total_mean_edge_count")

In [ ]:
count_edges_clusters['labels'].value_counts()

In [ ]:
count_edges_clusters.to_crs(crs_deg).explore(column="labels", categorical=True,cmap='viridis',\
                                               legend=True)

In [ ]:
#ax=count_edges_clusters.to_crs(crs_deg).plot(column="labels", categorical=True,cmap='viridis',\
 #                                              legend=True, figsize=(9, 9))
#cx.add_basemap(ax,crs=count_edges_clusters.to_crs(crs_deg).crs.to_string())

In [ ]:

#ax=count_edges_clusters.to_crs(crs_deg).plot(column="labels", categorical=True,cmap='viridis',\
#                                               legend=True, figsize=(9, 9))
#cx.add_basemap(ax,crs=count_edges_clusters.to_crs(crs_deg).crs.to_string())